# Dropout
: overfitting을 막기위한 방법으로 인공신경망의 뉴런을 몇개 제거

: 일반적으로 0.2~0.5 사용

: training단계에서만 사용

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

## Model with dropout

In [25]:
class DropoutNet(nn.Module):
    def __init__(self):
        super(DropoutNet,self).__init__()
        
        self.fc1 = nn.Linear(784, 500)
        self.dropout = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(500,10)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        return x
    
    """
    init: torch.Size([64, 1, 28, 28])
    view: torch.Size([64, 784])
    after fc1: torch.Size([64, 500])
    after dropout: torch.Size([64, 500])
    after fc1: torch.Size([64, 10])
        
    """

## Model without dropout

In [26]:
class NonDropout(nn.Module):
    def __init__(self):
        super(NonDropout, self).__init__()

        self.fc1 = nn.Linear(784, 500)
        self.fc2 = nn.Linear(500,10)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)

        return x

## result

In [27]:
# 모델 인스턴스 생성
model_dropout = DropoutNet()
model_nondropout = NonDropout()
# 출력 값 확인
print(model_dropout)
print(model_nondropout)

DropoutNet(
  (fc1): Linear(in_features=784, out_features=500, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)
NonDropout(
  (fc1): Linear(in_features=784, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)


## MNIST datasets downlaod

In [28]:
train_transform = transforms.Compose([
    transforms.AutoAugment(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.2,))
])
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.2,))
])


In [29]:
train_dataset = datasets.MNIST(root='./MNIST_data',
                              train=True,
                              transform=train_transform,
                              download=True)
test_dataset = datasets.MNIST(root='./MNIST_data',
                               train=False,
                               transform=test_transform)

In [30]:
BATCH_SIZE = 64
train_loader = DataLoader(dataset=train_dataset,
                              batch_size=BATCH_SIZE,
                              shuffle=True)

test_loader = DataLoader(dataset=test_dataset,
                              batch_size=BATCH_SIZE,
                              shuffle=False)

## Set loss, optimizer 

In [31]:
dropout_criterion = nn.CrossEntropyLoss()
dropout_optim = optim.SGD(model_dropout.parameters(), lr=0.001)

nondrop_criterion = nn.CrossEntropyLoss()
nondrop_optim = optim.SGD(model_nondropout.parameters(), lr = 0.001)

## DROPOUT train/test

In [36]:
#Train
for epoch in range(10):
    model_dropout.train()   #train mode 선언
    for images, labels in train_loader:
        dropout_optim.zero_grad()

        outputs = model_dropout(images)
        dropout_loss = dropout_criterion(outputs, labels)
        dropout_loss.backward()
        dropout_optim.step()
    
    if (epoch+1)%10 ==0:
        print(f"DROPOUT epoch: [{epoch+1}/10], Error: {dropout_loss.item()}")

#TEST
model_dropout.eval()
with torch.no_grad():
    dropout_corr = 0
    dropout_total = 0
    for images, labels in test_loader:
        test_out = model_dropout(images)
        _, dropout_pre = torch.max(test_out.data, 1)
        dropout_total += images.size(0)
        dropout_corr += (dropout_pre == labels).sum().item()

    print(f"DROPOUT accuaracy: {100* dropout_corr/dropout_total :.2f}")


DROPOUT epoch: [10/10], Error: 0.528876543045044
DROPOUT accuaracy: 93.83


## NONDROPOUT trian/test

In [35]:
#Train
for epoch in range(10):
    model_nondropout.train()   #train mode 선언
    for images, labels in train_loader:
        nondrop_optim.zero_grad()

        non_outputs = model_nondropout(images)
        nondropout_loss = nondrop_criterion(non_outputs, labels)
        nondropout_loss.backward()
        nondrop_optim.step()

    if (epoch+1)%10 ==0:
        print(f"NONDROPOUT epoch: [{epoch+1}/10], Error: {nondropout_loss.item()}")

#TEST
model_nondropout.eval()
with torch.no_grad():
    nondropout_corr = 0
    nondropout_total = 0
    for images, labels in test_loader:
        non_test_out = model_nondropout(images)
        _, nondropout_pre = torch.max(non_test_out.data, 1)
        nondropout_total += images.size(0)
        nondropout_corr += (nondropout_pre == labels).sum().item()

    print(f"DROPOUT accuaracy: {100* nondropout_corr/nondropout_total :.2f}")


NONDROPOUT epoch: [10/10], Error: 0.5317409634590149
DROPOUT accuaracy: 91.53
